In [1]:
import mysql.connector
from datetime import datetime, timedelta

# ---------------- Database Setup ----------------
conn = mysql.connector.connect(
    host="localhost",
    user="root",          # your MySQL username
    password="Arvind@2003",  # your MySQL password
    database="healthcare_final"   # make sure this DB exists
)
cursor = conn.cursor()

# Create doctors table
cursor.execute('''CREATE TABLE IF NOT EXISTS doctors (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    name VARCHAR(100),
                    department VARCHAR(100),
                    specialty VARCHAR(100)
                )''')

# Create appointments table
cursor.execute('''CREATE TABLE IF NOT EXISTS appointments (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    patient_name VARCHAR(100),
                    date DATE,
                    time TIME,
                    symptoms VARCHAR(255),
                    doctor_name VARCHAR(100),
                    department VARCHAR(100)
                )''')

# Insert sample doctors if empty
cursor.execute("SELECT COUNT(*) FROM doctors")
if cursor.fetchone()[0] == 0:
    doctors_data = [
        ("Dr. Smith", "Cardiology", "chest pain"),
        ("Dr. Johnson", "Neurology", "headache"),
        ("Dr. Williams", "Dermatology", "skin rash"),
        ("Dr. Brown", "General Medicine", "fever"),
        ("Dr. Davis", "Orthopedics", "back pain"),
    ]
    cursor.executemany("INSERT INTO doctors (name, department, specialty) VALUES (%s, %s, %s)", doctors_data)
    conn.commit()

In [2]:
# ---------------- Helper Functions ----------------
def assign_doctor(symptoms):
    cursor.execute("SELECT name, department FROM doctors WHERE specialty LIKE %s", (f"%{symptoms}%",))
    result = cursor.fetchone()
    if result:
        return result
    else:
        cursor.execute("SELECT name, department FROM doctors WHERE department = 'General Medicine'")
        return cursor.fetchone()

def select_date():
    today = datetime.now().date()
    dates = [today + timedelta(days=i) for i in range(5)]
    print("\n📅 Choose appointment date:")
    for i, d in enumerate(dates, start=1):
        print(f"{i}. {d}")
    choice = int(input("Enter choice: "))
    return str(dates[choice-1])

def select_time():
    now = datetime.now()
    slots = [(now + timedelta(hours=i)).strftime("%H:%M") for i in range(1, 6)]
    print("\n⏰ Choose appointment time:")
    for i, t in enumerate(slots, start=1):
        print(f"{i}. {t}")
    choice = int(input("Enter choice: "))
    return slots[choice-1]

def select_symptom():
    symptoms_list = ["chest pain", "headache", "skin rash", "fever", "back pain"]
    print("\n🤒 Select your symptom:")
    for i, s in enumerate(symptoms_list, start=1):
        print(f"{i}. {s}")
    choice = int(input("Enter choice: "))
    return symptoms_list[choice-1]

def book_appointment():
    patient_name = input("Enter your name: ")
    date = select_date()
    time = select_time()
    symptoms = select_symptom()

    doctor, department = assign_doctor(symptoms)

    cursor.execute('''INSERT INTO appointments (patient_name, date, time, symptoms, doctor_name, department)
                      VALUES (%s, %s, %s, %s, %s, %s)''',
                   (patient_name, date, time, symptoms, doctor, department))
    conn.commit()

    print(f"\n✅ Appointment booked successfully!")
    print(f"Patient: {patient_name}\nDate: {date}\nTime: {time}")
    print(f"Symptoms: {symptoms}\nAssigned Doctor: {doctor} ({department})\n")
def view_appointments():
    patient_name = input("Enter your name to view appointments: ")
    cursor.execute("SELECT * FROM appointments WHERE patient_name = %s", (patient_name,))
    appointments = cursor.fetchall()

    if appointments:
        print("\n📋 Your Appointments:")
        for appt in appointments:
            print(f"ID: {appt[0]} | Date: {appt[2]} | Time: {appt[3]} | Symptoms: {appt[4]} | Doctor: {appt[5]} ({appt[6]})")
    else:
        print("\n⚠️ No appointments found.")

def delete_appointment():
    patient_name = input("Enter your name: ")
    cursor.execute("SELECT * FROM appointments WHERE patient_name = %s", (patient_name,))
    appointments = cursor.fetchall()

    if not appointments:
        print("\n⚠️ No appointments to cancel.")
        return

    print("\n❌ Select appointment to cancel:")
    for i, appt in enumerate(appointments, start=1):
        print(f"{i}. ID: {appt[0]} | Date: {appt[2]} | Time: {appt[3]} | Doctor: {appt[5]}")

    choice = int(input("Enter choice: "))
    appt_id = appointments[choice-1][0]

    cursor.execute("DELETE FROM appointments WHERE id = %s", (appt_id,))
    conn.commit()

    print("\n✅ Appointment cancelled successfully.")

In [3]:
# ---------------- Chatbot ----------------
def chatbot():
    print("🤖 Welcome to the Hospital Chatbot!")
    while True:
        print("\nWhat would you like to do?")
        print("1. Book Appointment")
        print("2. View Appointments")
        print("3. Cancel Appointment")
        print("4. Exit")

        choice = input("Enter choice (1-4): ")

        if choice == "1":
            book_appointment()
        elif choice == "2":
            view_appointments()
        elif choice == "3":
            delete_appointment()
        elif choice == "4":
            print("👋 Goodbye!")
            break
        else:
            print("⚠️ Invalid choice. Try again.")

In [14]:
# Run chatbot
chatbot()

🤖 Welcome to the Hospital Chatbot!

What would you like to do?
1. Book Appointment
2. View Appointments
3. Cancel Appointment
4. Exit
⚠️ Invalid choice. Try again.

What would you like to do?
1. Book Appointment
2. View Appointments
3. Cancel Appointment
4. Exit

⚠️ No appointments found.

What would you like to do?
1. Book Appointment
2. View Appointments
3. Cancel Appointment
4. Exit

📋 Your Appointments:
ID: 1 | Date: 2025-10-01 | Time: 17:49:00 | Symptoms: back pain | Doctor: Dr. Davis (Orthopedics)
ID: 2 | Date: 2025-10-02 | Time: 19:54:00 | Symptoms: skin rash | Doctor: Dr. Williams (Dermatology)
ID: 3 | Date: 2025-10-03 | Time: 16:59:00 | Symptoms: fever | Doctor: Dr. Brown (General Medicine)

What would you like to do?
1. Book Appointment
2. View Appointments
3. Cancel Appointment
4. Exit
👋 Goodbye!
